This notebook, written by E. Karlé, contains the code necessary to reproduce Table 1 and Figure 6a from the article Dynamic Ranking and Translation Synchronization https://arxiv.org/pdf/2207.01455.pdf

In [1]:
import os
import random
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

from datetime import datetime
from itertools import combinations

import sys
sys.path.append('modules')

import real_data_module as rdata
import graph_module as graph
import loocv_module as cv

graph


In [2]:
plt.rcParams.update({'font.size': 16})

To use this notebook, the user can download the data from http://www.netflixprize.com, which is provided as 4 txt files and one csv file containing names of the movies

In [3]:
# Path to the folder where the data is stored

dir_path = '/Users/eglantine.karle/Docs/These/Ranking/LS approach/Code/Revision/netflix_data/'

# Preparation of the data

This section of the code needs to be run the first time one uses the notebook. The prepared data is then automatically saved in the data directory.

In [4]:
# Combination of all data into one csv file
if not os.path.isfile(dir_path+'data.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in the dataset and store them in one big file ('data.csv')
    # We're reading from each of the four files and appending each rating to a global file 'data.csv'
    data = open(dir_path+'data.csv', mode='w')
    
    row = list()
    files = [
        dir_path+'combined_data_1.txt',
        dir_path+'combined_data_2.txt', 
        dir_path+'combined_data_3.txt', 
        dir_path+'combined_data_4.txt'
    ]
    for file in files:
        print("Reading ratings from {}\n".format(file))
        with open(file) as f:
            for line in f: 
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
    data.close()
    
# Creating the dataframe from data.csv file
df = pd.read_csv(dir_path+'data.csv', sep=',', 
    names=['movie', 'user', 'rating', 'date'])
df.date = pd.to_datetime(df.date)
df.date = df['date'].dt.to_period('M')

# Arranging the ratings according to time-stamp(s)
df.sort_values(by='date', inplace=True)

In [5]:
# Get all the time stamps
dates = df.date.unique()

In [6]:
# Get all the movie ids
movie_id = pd.read_csv(dir_path+'movie_titles.csv',sep=';',header=None)
movie_id = movie_id.iloc[:,[0,1]]
movie_id.columns = ['Year','Movie'] 
movie_id.head()

,Year,Movie
0,2003.0,Dinosaur Planet
1,2004.0,Isle of Man TT 2004 Review
2,1997.0,Character
3,1994.0,Paula Abdul's Get Up & Dance
4,2004.0,The Rise and Fall of ECW


## Extraction of $N=100$ movies

The original dataset contains 17770 movies. For computational reasons, we extract a subset of 100 movies. We select first the 25 movies that have been rated at all of the 74 time points. We then add 75 random other movies.

In [7]:
# Selection of 100 movies

df_rated_movies = df.loc[df.date == '1999-11'] # Movies rated at the first time point
l_movies = []
for m in df_rated_movies.movie.unique():
    df_movie = df.loc[df.movie == m]
    if len(df_movie.date.unique()) == 74: # Add movies observed at all times
        l_movies.append(m)
        
for m in df.movie.unique(): # Select other random movies to our list
    if len(l_movies)<100:
        if m not in l_movies:
            l_movies.append(m)

In [10]:
# Create the list of dates to merge in order to get all connected graphs
# We gather data chronologically into graphs. We start a new graph as soon as the previous graph is connected


N = 100
list_A = [np.zeros((N,N))] # list of adjacency matrix
df_100 = df.loc[df.movie.isin(l_movies)] # df of the selected 100 movies

k = 0
d = []
merged_dates = [dates[0]] # list of merged time points to form connected graphs
while k< len(dates):
    if graph.connected(list_A[-1]):
        # if the last graph is connected, we start a new graph using the data of the current graph
        A = np.zeros((N,N))
        d = [dates[k]] # time points used to form the current graph
        df_d = df_100.loc[df_100.date.isin(d)] # Dataset at time d
        l_movies_d = df_d.movie.unique() # list of rated movies at time d
        
        # Add 1 in adjacency matrix for the movies rated at time d 
        for [i,j] in combinations(l_movies_d,2):
            a = l_movies.index(i)
            b = l_movies.index(j)
            A[a,b] = 1
            A[b,a] = 1
        list_A.append(A)
        merged_dates.append(d)
            
    else:
        # if the last graph is not connected, we add the data of the current graph
        d.append(dates[k]) # time points used for this current graph
        A = np.zeros((N,N))
        df_d = df_100.loc[df_100.date.isin(d)] # Dataset at time d
        l_movies_d = df_d.movie.unique() # list of rated movies at time d
        # Add 1 in adjacency matrix for the movies rated at time d 
        for [i,j] in combinations(l_movies_d,2):
            a = l_movies.index(i)
            b = l_movies.index(j)
            A[a,b] = 1
            A[b,a] = 1
        list_A[-1] = A
        merged_dates[-1] = d
        
    k = k+1

In [ ]:
# Create the observation and adjacency matrix from the merged dates

N = 100 # Number of movies
T = len(merged_dates) # Number of time points (number of observed connected graphs)
Y = np.zeros((T,N,N))
A = np.zeros((T,N,N))
df_100 = df.loc[df.movie.isin(l_movies)]
for k,d in enumerate(merged_dates):
    df_d = df_100.loc[df_100.date.isin(d)] # Dataset at time d
    l_movies_d = df_d.movie.unique() # list of rated movies at time d
    for [i,j] in combinations(l_movies_d,2):
        a = l_movies.index(i)
        b = l_movies.index(j)
        
        # Mean score for each movie at time d
        rating_i = np.mean(df_d['rating'].values[df_d.movie == i])
        rating_j = np.mean(df_d['rating'].values[df_d.movie == j])
        # Update the observation matrix with the mean rating difference
        Y[k,a,b] = rating_i-rating_j
        Y[k,b,a] = -Y[k,a,b]
        # Update the adjacency matrix
        A[k,a,b] = 1 
        A[k,b,a] = 1

In [ ]:
# Save the prepared data

import pickle
with open("netflix_data/y_merged_transync_100.txt", "wb") as y:
    pickle.dump(Y, y)
with open("netflix_data/a_merged_transync_100.txt", "wb") as a:
    pickle.dump(A, a)

# Analysis of 100 movies

Once the data has been prepared once, the user can directly proceed with the analysis.

## Load the data

In [ ]:
import pickle

with open(dir_path+'y_merged_transync_100.txt', "rb") as y:
    Y = pickle.load(y) # Observation matrix
with open(dir_path+'a_merged_transync_100.txt', "rb") as a:
    A = pickle.load(a) # Adjacency matrix

In [ ]:
T,N = np.shape(Y)[:2] # Number of time points and of movies
T,N

As a sanity check, we verify that the union graph and all the individual graphs are connected

In [ ]:
import graph_module as graph
print(graph.connected(sum(A)))
for t in range(T):
    print(graph.connected(A[t,:,:]))

In this case, the union graph is connected and all the merged graphs are connected.
We've reduced the number of timepoints to 23 to obtain this connectivity

In [ ]:
# Check sparsity of the graphs

sparsity = []
for t in range(T):
    sparsity.append(1.0 - ( np.count_nonzero(A[t,:,:]) / float(A[t,:,:].size) ))
sparsity

The observed graph here are dense

## Analysis via DLS and DProj method

In [ ]:
import loocv_module as cv
import tools_module as tools
import smoothness_module as smooth

In [ ]:
# Smoothness parameters used for the analysis

E = smooth.penalty_E(N,T)
eigs_E,V_E = smooth.eigs_E(N,T-1) # eigs_E compute the eigs of the smoothness operator for T+1 graphs

## Choice of $\lambda^*$ and $\tau^*$ through cross validation

We run two cross-validations procedures in order to get optimal values for the hyper parameter $\lambda$ and $\tau$. The criteria for these procedures are the MSE and the mean number of upsets.

In [ ]:
# Cross Validation with the number of upsets criterion
random.seed(0)
np.random.seed(0)

lambda_list_up = np.linspace(0,0.2,20) # Candidates for lambda
tau_list_up = np.linspace(370,410,20) # Candidates for tau

# Analysis with the DLS method
lam_up,z_up_dls = cv.cv_dls_up(Y,A,E,lambda_list_up,num_loocv = 40)

# Analysis with the DProj method
tau_up,z_up_dproj = cv.cv_dproj_up(Y,A,V_E,eigs_E,tau_list_up,num_loocv = 40)

In [ ]:
# Cross Validation with MSE criterion
random.seed(0)
np.random.seed(0)

lambda_list_mse = np.linspace(0,500,20) # Candidates for lambda
tau_list_mse = np.linspace(1e-6,1,20) # Candidates for tau

# Analysis with the DLS method
lam_mse,z_mse_dls = cv.cv_dls_mse(Y,A,E,lambda_list_mse,num_loocv = 40)

# Analysis with the DProj method
tau_mse,z_mse_dproj = cv.cv_dproj_mse(Y,A,V_E,eigs_E,tau_list_mse,num_loocv = 40)

In [ ]:
print(lam_up,lam_mse,tau_up,tau_mse)

The cross-validation procedures give estimators for the optimal value of hyperparameters in DLS and DProj method. Let us compute the naive LS estimator for the sake of comparison.

In [ ]:
# Computation of the LS estimator

Y_vec = tools.obs_transync(Y,A) # Vectorize the observations
Q = graph.diag_incidence(A)
Lv = Q@Q.T # Laplacian matrix
z_ls = scipy.sparse.linalg.lsqr(Lv,Q@Y_vec)[0] # LS estimator
z_ls = z_ls.reshape((T,N),order='F')

Let us now compute the error criterion for each estimator using the observations as ground truth. 
For estimators obtained by cross-validation with the Upsets criterion, we compute the mean number of upsets with respect to the observations.
For estimators obtained by cross-validation with the MSE criterion, we compute the MSE with respect to the observed score differences.

In [ ]:
# Mean Number of upsets 
upsets_ls = rdata.get_mean_nb_upsets(Y,A,z_ls)
upsets_dls = rdata.get_mean_nb_upsets(Y,A,z_up_dls)
upsets_dproj = rdata.get_mean_nb_upsets(Y,A,z_up_dproj)

print(upsets_ls,upsets_dls,upsets_dproj)

In [ ]:
# MSE
MSE_ls = rdata.get_mse(Y,A,z_ls)
MSE_dls = rdata.get_mse(Y,A,z_mse_dls)
MSE_dproj = rdata.get_mse(Y,A,z_mse_dproj)

print(MSE_ls,MSE_dls,MSE_dproj)

# Sanity check : smoothness of the data

Our analysis rely on a supposed smoothness of the data. Let us check that this dataset fits this criteria by defining a ground truth vector from the observations and plot its evolution for some movies.

We define the ground truth vector $z^*$ such that 
$$z^*_{t,i} = \frac{1}{N_{t,i}} \sum_{j \in N_{t,i}} y_{ij}(t)$$
where $N_{t,i}$ denotes the set of neighbours of $i$ at time $t$

In [ ]:
# Define ground truth
z_star = np.zeros((T,N))
for t in range(T):
    for i in range(N):
        Nti = np.sum(A[t,i,:]) # Number of games played by team i at time t
        if Nti != 0:
            z_star[t,i] = np.sum(Y[t,i,:])/Nti
        else:
            z_star[t,i] = 0

In [ ]:
# Plot for 5 movies
fig,ax = plt.subplots(figsize=(8,6))
for i in range(5):
    ax.plot(z_star[:,i],label=df_movies.iloc[i]['Movies'])
ax.set_ylabel('Ground truth $z^{*,emp}_{t,i}$')
ax.set_xlabel('Time')
ax.set_title('Evolution of the strength of movies')
plt.legend(loc='best',frameon=False)

plt.show()